### Import Libraries

In [1]:
import cv2
import mediapipe as mp
import pygame
import sys
import random
import numpy as np

pygame 2.6.1 (SDL 2.28.4, Python 3.9.21)
Hello from the pygame community. https://www.pygame.org/contribute.html


### MediaPipe Detector

In [2]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose() #initializing pose object from mediapipe for detection

### Pygame Initialization

In [3]:
pygame.init() #initializing pygame modules
screen=pygame.display.set_mode((800, 600)) #dimension of game window 
pygame.display.set_caption("Simran's Office Paper Toss - Simulated Experience")
clock=pygame.time.Clock()

In [4]:
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
WHITE=(255,255,255)
BLACK=(0,0,0)

In [5]:
basket_img=pygame.Rect(700,450,80,80) #loading basket position and size

In [6]:
start_game_flag=False

### Wrist Choice

In [7]:
selected_wrist = input("Choose which wrist to play with (left/right): ").strip().lower()
if selected_wrist not in ['left', 'right']:
    print("Invalid choice. Will default to using right wrist.")
    selected_wrist = 'right'

Choose which wrist to play with (left/right):  right


### Webcam and Motion Track

In [8]:
cam=cv2.VideoCapture(0) #start webcam

In [9]:
prev_wrist=None #previous wrist motion 
def wrist_mov():
    global prev_wrist
    global selected_wrist
    ret, frame=cam.read() #frame of camera 
    if not ret:
        return None, None, None
    frame=cv2.flip(frame, 1)
    rgb=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results=pose.process(rgb) #detect pose landmarks using the pose model 

    wrist_pos=None #initializing starting wrist position and velocity 
    velocity=np.array([0.0,0.0], dtype=np.float64)

    if results.pose_landmarks:
        if selected_wrist == 'right':
            wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
        else:
            wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

        wrist_pos=np.array([wrist.x*800, wrist.y*600]) #scaling to screen size of current wrist motion
        #draw landmarks and connections on frame 
        mp.solutions.drawing_utils.draw_landmarks(
            frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=4),
            connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(0,0,255), thickness=2)
        )

        cx, cy = int(wrist.x * frame.shape[1]), int(wrist.y * frame.shape[0]) #indicates wrist position on frame 
        cv2.circle(frame, (cx, cy), 10, (255, 0, 0), -1)
        cv2.putText(frame, 'Wrist Movement', (cx+10, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

        if prev_wrist is not None: #difference in velocity between previous wrist and current wrist motion
            velocity=wrist_pos - prev_wrist
        else:
            velocity=np.array([0.0,0.0], dtype=np.float64)#set velocity to 0 if there is no previous wrist motion

        prev_wrist=wrist_pos #updating previous wrist motion for next iteration
        return wrist_pos, velocity, frame
    return None, None, frame

### Paper Throwing

In [10]:
class PaperBall:
    def __init__(self, pos):  # initial position
        self.initial_pos = np.array(pos, dtype=np.float64)  # stores original
        self.pos = self.initial_pos.copy()
        self.vel = np.array([0.0, 0.0], dtype=np.float64)
        self.active = False  # wait to throw paper ball
    def throw(self, velocity):
        #velocity=np.array(velocity, dtype=np.float64)
        self.vel=np.array(velocity, dtype=np.float64)*1.2 #scale speed
        self.active=True
    def update(self):
        if self.active:
            #print(f"Pos Type: {self.pos.dtype}, Vel Type: {self.vel.dtype}")  # Check types
            self.pos += self.vel
            self.vel[1] += 0.5  # accounting for gravity
        if self.pos[1] > 600.0 or self.pos[0] > 800.0 or self.pos[0] < 0.0:
            self.active = False
            self.pos = self.initial_pos.copy()# reset position
            self.vel = np.array([0.0, 0.0], dtype=np.float64)# reset velocity     
    def draw(self,screen):
        pygame.draw.circle(screen, (180,180,180), self.pos.astype(int), 15)

### Basket

In [11]:
class Basket:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.width = 80
        self.height = 40
        self.rect = pygame.Rect(self.x, self.y, self.width, self.height)
        self.scored_balls=[] # acts as storage unit of landing paper 
    def draw(self, screen):
        pygame.draw.rect(screen, (160,82,45), (self.x, self.y - 10, self.width, 20)) #outline of basket
        pygame.draw.ellipse(screen, (160, 82, 45), (self.rect.x, self.rect.y, self.width, self.height)) #rim of basket
        pygame.draw.rect(screen, (210, 180, 140), (self.rect.x, self.rect.y+self.height-10, self.width, 20), 1) #basket body

        #--- stacking the balls---
        for i, ball in enumerate(self.scored_balls):
            stack_x=self.x+self.width//2
            stack_y=self.y+self.height-10-i*12 
            pygame.draw.circle(screen, (180, 180, 180), (stack_x,stack_y), 10)
                                                #---Successfull collision between ball and basket---
    def collision_check(self, paper_ball):
        bx, by=paper_ball.pos
        return self.rect.collidepoint(bx,by)
    def add_scored_paper(self, paper_ball):
        self.scored_balls.append(paper_ball.pos.copy())
    def random_move(self, screen_width, screen_height, margin=100, ui_height=80):
        max_x = screen_width - self.width - margin
        max_y = screen_height - self.height - margin
        min_x = margin
        min_y = ui_height 
        self.x = random.randint(min_x, max_x)
        self.y = random.randint(min_y, max_y)
        self.rect.topleft = (self.x, self.y)

### Quit Function

In [12]:
button_rect = pygame.Rect(650, 20, 120, 40)  
button_color = (200, 0, 0)
button_text = "Quit"

### Main Game

In [13]:
#initialize paper ball 
if selected_wrist == 'right':
    basket_x = 100   # basket on left
    ball_start_x = 600  # ball on right
else:
    basket_x = 600   # basket on right
    ball_start_x = 100  # ball on left

basket = Basket(basket_x, 500)
paper_throw = PaperBall(pos=[ball_start_x, 500])
basket = Basket(basket_x, 500)
base_score=0

In [14]:
running=True
game_started = False
prev_wrist_pos = None
cooldown_counter = 0
throw_cooldown = 20
velocity_threshold = 15  
while running: 
    screen.fill((255, 255, 255)) #clear screen 
    basket.draw(screen)
    wrist_pos, velocity, frame=wrist_mov() #refer to wrist movement function

                                                                 #--- Start Game---
    if not start_game_flag:
        pygame.draw.rect(screen, (0, 128, 255), (300, 250, 200, 60))  
        font = pygame.font.SysFont(None, 40)
        text = font.render('Start Game', True, (255, 255, 255))
        screen.blit(text, (320, 265))
    
    if game_started and wrist_pos is not None:
        if prev_wrist_pos is not None:
            velocity = np.array(wrist_pos) - np.array(prev_wrist_pos)
        if selected_wrist == 'right':
            velocity[0] *= -1  # flip direction for mirrored motion

        speed = np.linalg.norm(velocity)

        if speed > velocity_threshold and cooldown_counter <= 0 and not paper_throw.active:
            paper_throw.throw(velocity)
            cooldown_counter = throw_cooldown

    prev_wrist_pos = wrist_pos

    if cooldown_counter > 0:
        cooldown_counter -= 1

    #update and draw paper object
    paper_throw.update() #update and draw the paper 
    paper_throw.draw(screen)
    
    #show collision 
    if paper_throw.active and basket.collision_check(paper_throw):
        base_score+=1 #incrementing score for every collision 
        basket.add_scored_paper(paper_throw) #visually adds paper to the basket by reference to add_scored_paper function 
        paper_throw.active=False #deactivate the paper once thrown and added to the basket 
        paper_throw.pos=np.array([100.0,500.0], dtype=np.float64)
        paper_throw.vel=np.array([0,0]) #resetting velocity 
        basket.random_move(screen_width, screen_height) #move the basket
        
    #show score
    font=pygame.font.SysFont(None, 40)
    text=font.render(f"Player's Score: {base_score}", True, BLACK)
    screen.blit(text, (20,20))

                                                                #---Quit button creation---
    pygame.draw.rect(screen, button_color, button_rect) #defining quit button on the screen 
    font = pygame.font.SysFont(None, 30)
    text_surface = font.render(button_text, True, (255, 255, 255))
    text_rect = text_surface.get_rect(center=button_rect.center)  
    screen.blit(text_surface, text_rect) #drawing text on the button 

                                                                    #--- Camera Feed ---

    if frame is not None:
        cv2.imshow("Tracking Movement", frame)

                                                                #--- Quitting the game---
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        elif event.type == pygame.MOUSEBUTTONDOWN:
            if button_rect.collidepoint(event.pos):
                running = False  # Quit button
            
            elif not game_started and pygame.Rect(300, 250, 200, 60).collidepoint(event.pos):
                game_started = True
                start_game_flag= True
                paper_throw.active = False
                base_score = 0
                cooldown_counter = 0
                prev_wrist_pos = None


    if cv2.waitKey(1) &0xFF==ord('q'): #quits game on 'q' click
        break

    pygame.display.flip()
    clock.tick(30) #30 frames per second 
    
cam.release() #release camera after quit 
pygame.quit()
sys.exit()

SystemExit: 

C:\Users\svkar\anaconda3\envs\office-sim\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
